# Linear / Ridge / Lasso Regression

## Mục tiêu
- Xây dựng mô hình Linear / Ridge / Lasso Regression để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 

## Giới thiệu mô hình

Linear Regression là mô hình hồi quy tuyến tính cổ điển, mô tả mối quan hệ giữa biến đầu vào và đầu ra dưới dạng phương trình tuyến tính.

### **Ưu điểm:**

- Dễ hiểu, dễ triển khai.

- Huấn luyện rất nhanh.

- Phù hợp khi quan hệ gần tuyến tính.

### **Nhược điểm:**

- Không phù hợp cho dữ liệu phi tuyến.

- Dễ bị ảnh hưởng bởi hiện tượng đa cộng tuyến.

- Nhạy cảm với outlier.

## Bước 1 - Import các thư viện cần thiết

### 1.1. Import thư viện

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

RANDOM_STATE = 42

### 1.2. Cấu hình thư mục

In [2]:
os.makedirs("../models/1_linear_regression", exist_ok=True)

## Bước 2 - Đọc dữ liệu đã tiền xử lý
Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [3]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (dữ liệu đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 13)

5 dòng đầu tiên của tập train (dữ liệu đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Romania,ROU,2019,-0.100387,-1.101137,75.607317,-0.107887,0.079336,1.170060,0.621932,0.746494,-0.047183,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.500838,72.432195,-0.405891,0.241659,-0.060699,0.586683,0.718065,-0.221970,-0.189257
2,Angola,AGO,2008,-0.083899,1.500859,55.281000,-0.470953,1.327705,-0.060699,-1.545881,-2.268636,-0.432669,1.598947
3,Albania,ALB,2001,-0.222240,-1.349940,75.083000,-0.584364,0.930412,-1.071878,0.600783,-0.024271,-0.404910,-0.104497
4,Central African Republic,CAF,2021,-0.206911,0.236338,40.279000,-0.618434,-0.427187,-1.640089,-2.349558,-3.008624,-0.529029,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [4]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3255, 10)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình Linear Regression 


### 4.1. Fit mô hình LinearRegression() trên tập train.

In [5]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_lin = linear_model.predict(X_train)

### 4.2. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [6]:
# Tính 3 loại loss: MAE, RMSE và R2
lin_rmse = np.sqrt(mean_squared_error(y_train, y_pred_lin))
lin_r2  = r2_score(y_train, y_pred_lin)
lin_mae = mean_absolute_error(y_train, y_pred_lin)

print("KẾT QUẢ MÔ HÌNH")
print("="*60)
print("Model: LinearRegression:")
print(f"RMSE loss: {lin_rmse:.3f}")
print(f"MAE loss: {lin_mae:.3f}")
print(f"R2 score: {lin_r2:.3f}")

# Lưu mô hình
joblib.dump(linear_model, "../models/1_linear_regression/linear.pkl")

KẾT QUẢ MÔ HÌNH
Model: LinearRegression:
RMSE loss: 4.259
MAE loss: 3.170
R2 score: 0.759


['../models/1_linear_regression/linear.pkl']

## Bước 5 - Xây dựng và huấn luyện mô hình Ridge
### Mục tiêu:
- Tối ưu hóa siêu tham số Ridge bằng RandomizedSearchCV (k=5)


### 5.1. Huấn luyện mô hình sử dụng phương pháp k-fold cross-validation (k=5) 

In [7]:
from scipy.stats import loguniform

param_distributions_ridge = {
    "alpha": loguniform(1e-4, 1e3), 
    "fit_intercept": [True, False],
    "solver": ["auto", "svd", "lsqr", "sag", "saga"],   
    "tol": loguniform(1e-6, 1e-2),    
}

ridge = Ridge(random_state=RANDOM_STATE)
ridge_cv = RandomizedSearchCV(
    ridge, 
    param_distributions_ridge, 
    scoring='neg_mean_squared_error', 
    cv=5, 
    n_jobs=-1, 
    verbose=2,
    n_iter=60
)

# Huấn luyện mô hình
ridge_cv.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


,estimator,Ridge(random_state=42)
,param_distributions,"{'alpha': <scipy.stats....002B732C3A4D0>, 'fit_intercept': [True, False], 'solver': ['auto', 'svd', ...], 'tol': <scipy.stats....002B7311002D0>}"
,n_iter,60
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


### 5.2. Hiển thị siêu tham số tối ưu

In [8]:
best_ridge = ridge_cv.best_estimator_
best_params_svr = ridge_cv.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params_svr.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  alpha               : 15.778089825391449
  fit_intercept       : False
  solver              : auto
  tol                 : 2.1119952234897955e-06


### 5.3. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [9]:
# Tính 3 loại loss: MAE, RMSE và R2
y_pred_ridge = best_ridge.predict(X_train)
ridge_rmse = np.sqrt(mean_squared_error(y_train, y_pred_ridge))
ridge_r2  = r2_score(y_train, y_pred_ridge)
ridge_mae = mean_absolute_error(y_train, y_pred_ridge)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Ridge Regression:")
print(f"RMSE loss: {ridge_rmse:.3f}")
print(f"MAE loss: {ridge_mae:.3f}")
print(f"R2 score: {ridge_r2:.3f}")

# Lưu mô hình
joblib.dump(best_ridge, f"../models/1_linear_regression/ridge.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Ridge Regression:
RMSE loss: 4.259
MAE loss: 3.167
R2 score: 0.759


['../models/1_linear_regression/ridge.pkl']

## Bước 6 - Xây dựng và huấn luyện mô hình Lasso
### Mục tiêu:
- Tối ưu hóa siêu tham số Lasso bằng RandomizedSearchCV (k=5)


### 6.1. Huấn luyện mô hình sử dụng phương pháp k-fold cross-validation (k=5) 

In [10]:
param_distributions_lasso = {
    "alpha": loguniform(1e-4, 1e2),   # 0.0001 → 100 (Lasso nhạy hơn)
    "fit_intercept": [True, False],
    "selection": ["cyclic", "random"],   # random thường tốt hơn
    "max_iter": [1000, 2000, 5000, 10000],
    "tol": loguniform(1e-6, 1e-2),
}

lasso = Lasso(random_state=RANDOM_STATE, max_iter=5000)
lasso_cv = RandomizedSearchCV(
    lasso, 
    param_distributions_lasso, 
    scoring='neg_mean_squared_error', 
    cv=5, 
    n_jobs=-1,
    n_iter=60,
    verbose=2
)

lasso_cv.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


,estimator,Lasso(max_ite...ndom_state=42)
,param_distributions,"{'alpha': <scipy.stats....002B732C5EDD0>, 'fit_intercept': [True, False], 'max_iter': [1000, 2000, ...], 'selection': ['cyclic', 'random'], ...}"
,n_iter,60
,scoring,'neg_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


### 6.2. Hiển thị siêu tham số tối ưu

In [11]:
best_lasso = lasso_cv.best_estimator_
best_params_lasso = lasso_cv.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params_lasso.items():
    print(f"  {param:20s}: {value}")

SIÊU THAM SỐ TỐI ƯU
  alpha               : 0.010828637550147454
  fit_intercept       : False
  max_iter            : 5000
  selection           : random
  tol                 : 0.00065179673964859


### 6.3. In ra loss (MAE, RMSE và R2) và lưu mô hình

In [12]:
# Tính 3 loại loss: MAE, RMSE và R2
y_pred_lasso = best_lasso.predict(X_train)
lasso_rmse = np.sqrt(mean_squared_error(y_train, y_pred_lasso))
lasso_r2  = r2_score(y_train, y_pred_lasso)
lasso_mae = mean_absolute_error(y_train, y_pred_lasso)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Lasso Regression:")
print(f"RMSE loss: {lasso_rmse:.3f}")
print(f"MAE loss: {lasso_mae:.3f}")
print(f"R2 score: {lasso_r2:.3f}")

# Lưu mô hình
joblib.dump(best_lasso, f"../models/1_linear_regression/lasso.pkl")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Lasso Regression:
RMSE loss: 4.259
MAE loss: 3.168
R2 score: 0.759


['../models/1_linear_regression/lasso.pkl']

## Kết luận
Tổng kết quá trình huấn luyện 3 mô hình (Linear, Ridge, Lasso)
- Đã huấn luyện thành công các mô hình Linear/Ridge/Lasso"
- Phương pháp 5-fold CV được dùng để chọn siêu tham số cho Ridge & Lasso